<h3>Vào thư mục search_engine để lấy preprocessor</h3>

In [2]:
%cd search_engine/
from preprocess import Preprocessor # type: ignore


d:\Projects\Python\OOP-PROJECT-G4\src\main\python\search_engine


<h3>Import các thư viện</h3>

In [3]:
import pandas as pd # type: ignore
from gensim.models.doc2vec import Doc2Vec # type: ignore
from sklearn.metrics.pairwise import cosine_similarity
from joblib import dump, load
from sklearn.feature_extraction.text import TfidfVectorizer


<h3>Về thư mục gốc</h3>

In [4]:
%cd ../

d:\Projects\Python\OOP-PROJECT-G4\src\main\python


<h3>Load model Doc2Vec của gensim và gọi Preprocessor</h3>

In [5]:
model = Doc2Vec.load('search_engine/models/doc2vec_model.model')

preprocessor = Preprocessor()



<h3>Tải file CSV</h3>

In [6]:
relative_path = "../resources/data/data.csv"

sample_df = pd.read_csv(relative_path)

df = sample_df
df.head()

,Article link,Website source,Article type,Article title,Content,Creation date,Author,Category,Tags,Summary,Temp
0,https://www.theblock.co/post/285730/custodia-i...,https://www.theblock.co,News Article,Custodia is not entitled to a Fed master accou...,The Federal Reserve does not have to give digi...,"March 29, 2024, 7:05PM EDT",Sarah Wynn,Policy,COURT HEARINGS-LAWSUITS,Custodia Bank sued the central bank in 2022 fo...,NaN
1,https://www.theblock.co/post/285724/multicoin-...,https://www.theblock.co,News Article,"Multicoin Capital's hedge fund has grown 9,281...",Multicoin Capital’s crypto-focused hedge fund ...,"March 29, 2024, 7:00PM EDT UPDATED: March 29, ...",Elizabeth Napolitano,Companies,INVESTMENT FIRMS,"Multicoin Capital’s hedge fund has returned 9,...",NaN
2,https://www.theblock.co/post/285702/1kx-raise-...,https://www.theblock.co,News Article,1kx raises $75 million in latest funding round,"1kx has raised $75 million, the latest sign in...","March 29, 2024, 3:20PM EDT",Elizabeth Napolitano,Companies,NaN,Investment firm 1kx has raised $75 million for...,NaN
3,https://www.theblock.co/post/285690/cftc-commi...,https://www.theblock.co,News Article,CFTC Commissioner Pham says agency may be infr...,One of the Commodity Futures Trading Commissio...,"March 29, 2024, 12:06PM EDT",Sarah Wynn,Exchanges,CFTC-SEC,The agency’s complaint “appears to assert that...,NaN
4,https://www.theblock.co/post/285608/bitcoin-fu...,https://www.theblock.co,News Article,Bitcoin futures open interest reaches new high...,Open interest for bitcoin futures on centraliz...,"March 29, 2024, 11:03AM EDT UPDATED: March 29,...",Vishal Chawla,The Block,NaN,Bitcoin futures open interest on centralized e...,NaN


In [30]:
def search(query):
    query = preprocessor.preprocess_text(query)
    inferred_vector = model.infer_vector(query.split())
    
    sims = model.dv.most_similar([inferred_vector], topn=10)

    results = []
    for sim in sims:
        doc_index = int(sim[0])
        similarity = sim[1]
        title = df.iloc[doc_index][' Article title']
        content = df.iloc[doc_index][' Content']
        results.append((doc_index, similarity, title, content))

    return results



In [36]:
query = "nft"

for doc_index, similarity, title, content in search(query):
    print(doc_index)
    print(similarity)
    print(title)
    print(content)


1696
0.7723720669746399
Blur dominates almost 80% of NFT trading volume as market activity spikes
The NFT market has seen a spike in activity over the past month, with the majority of the increased trading volume occurring on the Blur BLUR +2.40% marketplace. According to The Block's Data Dashboard, November saw $605 million in Ethereum-based NFT trading. That's up from a monthly trading volume of $306 million in October, an increase of almost 100%. Just four days into December, NFT trading volume for the month has already reached almost $90 million, with nearly $70 million coming from transactions on the Blur NFT marketplace. Shift from OpenSea Since February of this year, the NFT sector has witnessed a notable market share shift from OpenSea to Blur, with the later marketplace now dominating almost 80% of total trading volume in the Ethereum-based NFT market. OpenSea, the NFT marketplace that was once dominant, has been ceding ground and currently holds around only 17% of NFT trading

In [9]:
tfidf_matrix = load("search_engine/models/tfidf/tfidf_matrix.joblib")
vectorizer = load("search_engine/models/tfidf/vectorizer.joblib")

In [10]:
def tfidf_query(query):
  preprocessed_query = preprocessor.preprocess_text(query)

# Tìm kiếm và xác định hàng liên quan nhất
  query_vector = vectorizer.transform([preprocessed_query])
  similarities = cosine_similarity(query_vector, tfidf_matrix)

  # Bước 6: Sắp xếp và hiển thị kết quả
  results = []
  for idx, sim in enumerate(similarities[0]):
      results.append((df.iloc[idx][' Article title'], sim))

  results.sort(key=lambda x: x[1], reverse=True)

  for result in results[:10]:
      print(result)

In [14]:
query = "bitcoin rise"
tfidf_query(query)

('Bitcoin, ether rise to close week as altcoins tick higher', 0.39048505235860403)
('Moonbirds creator Kevin Rose hacked, at least $1 million in NFTs stolen', 0.33108120217306314)
('Bitcoin rises past $45,000 amid US spot bitcoin ETF approval anticipation', 0.3148615400418044)
('Crypto market continues to rise, while Silvergate shares seesaw after probe report', 0.3135488098093932)
('Bitcoin price rises above $47,000 after SEC approval of spot BTC ETFs', 0.30740423363914404)
('Coinbase stock price rises to highest value since May 2022', 0.30524759484625974)
('This week in markets: Bitcoin breaks $20,000 as altcoins, crypto stocks surge', 0.30369991826354015)
('Bitcoin price rises back above $42,000, causing spike in short liquidations', 0.30238873435709474)
("Bitcoin rises above $27,000 ahead of Friday's expiring options", 0.3016264134620358)
('Bitcoin flirts with trillion dollar valuation as price rises above $47,000', 0.29919080925300023)
